## Tolman-Oppenheimer-Volkoff equations

*This worksheet illustrates some features of [SageManifolds](http://sagemanifolds.obspm.fr/) (v0.9) on the derivation of the Tolman-Oppenheimer-Volkoff equations (spherically symmetric, stationary solution of general relativity).*

We will calculate the Einstein equations   
$$R_{\mu\nu} - \frac{1}{2}Rg_{\mu\nu} = T_{\mu\nu}$$
for a corresponding spherically symmetric, stationary metric $g$. In the above, $R_{\mu\nu}$ is the Ricci tensor, $R=R^\mu_\mu$ is the Ricci scalar, and $T_{\mu\nu}$ is the energy-momentum tensor (left side of Einstein's equations describe the geometry of spacetime, and the right side the matter in the spacetime). 

In [1]:
%display text latex
set_nproc_tensor()
omit_function_args(True)

We first declare the spacetime `M` as a general 4-dimensional manifold,

In [2]:
M = DiffManifold(4, 'M')

with the standard spherical coordinates (`X` denotes the coordinate chart on `M`):

In [3]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta phi:(0,2*pi):\phi')

In order to define a general spherically symmetric, stationary metric one needs a few auxiliary functions of the radial coordinate $r$ - metric functions $\nu(r)$ and $\lambda(r)$, matter pressure $p(r)$ and energy density $\rho(r)$, as well as the mass $m(r)$ enclosed within the sphere of the radius $r$:  

In [4]:
# metric functions: 
nu = function("nu", r, latex_name=r"\nu")
lam = function("lambda", r, latex_name=r"\lambda")
# density and pressure: 
rho = function("rho", r, latex_name=r"\rho")
p = function("P", r)
# mass enclosed in sphere of radius r: 
m = function("m", r)

In general, such metric reads as follows: 

In [5]:
g = M.lorentz_metric('g')
g[0,0] = -exp(2*nu)
g[1,1] = exp(2*lam)
g[2,2], g[3,3] = r^2, r^2*sin(th)^2
g.display()

g = -e^(2*nu) dt*dt + e^(2*lambda) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dphi*dphi

which works assuming that the physical constants $G=c=1$. Let's introduce $G$ and $c$ as variables to obtain the dimensional version of the equations:

In [6]:
var('G c pi'); assume(G>0); assume(c>0)

From the Newtonian weak field limit considerations (Newtonian force far from the source) one may simplify the above expression and replace $\lambda(r)$ with $\frac{1-2Gm}{rc^2}$, as well as explicitly put $c^2$ in front of $g_{tt}$. Then

In [7]:
g[0,0] = -c^2*exp(2*nu)
g[1,1] = 1/(1-2*G*m/(r*c^2))
g.display()

g = -c^2*e^(2*nu) dt*dt + c^2*r/(c^2*r - 2*G*m) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dphi*dphi

The Ricci tensor is a result of a method `ricci()` acting on the metric `g`:

In [8]:
Ricci = g.ricci(); Ricci.display()

Ric(g) = (c^2*r^2*e^(2*nu)*(d(nu)/dr)^2 + c^2*r^2*e^(2*nu)*d^2(nu)/dr^2 + 2*c^2*r*e^(2*nu)*d(nu)/dr - (2*r*e^(2*nu)*m*(d(nu)/dr)^2 + 2*r*e^(2*nu)*m*d^2(nu)/dr^2 + (r*e^(2*nu)*d(m)/dr + 3*e^(2*nu)*m)*d(nu)/dr)*G)/r^2 dt*dt - (c^2*r^3*(d(nu)/dr)^2 + c^2*r^3*d^2(nu)/dr^2 - (2*r^2*m*(d(nu)/dr)^2 + 2*r^2*m*d^2(nu)/dr^2 + 2*r*d(m)/dr + (r^2*d(m)/dr - r*m)*d(nu)/dr - 2*m)*G)/(c^2*r^3 - 2*G*r^2*m) dr*dr - (c^2*r^2*d(nu)/dr - (2*r*m*d(nu)/dr + r*d(m)/dr + m)*G)/(c^2*r) dth*dth - (c^2*r^2*sin(th)^2*d(nu)/dr - (2*r*m*d(nu)/dr + r*d(m)/dr + m)*G*sin(th)^2)/(c^2*r) dphi*dphi

For example, the $R_{tt}$ component is 

In [9]:
Ricci[0,0]

(c^2*r^2*e^(2*nu)*(d(nu)/dr)^2 + c^2*r^2*e^(2*nu)*d^2(nu)/dr^2 + 2*c^2*r*e^(2*nu)*d(nu)/dr - (2*r*e^(2*nu)*m*(d(nu)/dr)^2 + 2*r*e^(2*nu)*m*d^2(nu)/dr^2 + (r*e^(2*nu)*d(m)/dr + 3*e^(2*nu)*m)*d(nu)/dr)*G)/r^2

In [10]:
Ricci[0,0].expand().collect(nu.diff(r)).collect(nu.diff(r,r)).collect(c^2*exp(2*nu)).collect_common_factors()

(c^2*(d(nu)/dr)^2 - 2*G*m*(d(nu)/dr)^2/r + c^2*d^2(nu)/dr^2 + 2*c^2*d(nu)/dr/r - G*d(m)/dr*d(nu)/dr/r - 2*G*m*d^2(nu)/dr^2/r - 3*G*m*d(nu)/dr/r^2)*e^(2*nu)

In [11]:
Ricci[1,1].expand().collect_common_factors()

-(c^2*r^3*(d(nu)/dr)^2 - 2*G*r^2*m*(d(nu)/dr)^2 + c^2*r^3*d^2(nu)/dr^2 - G*r^2*d(m)/dr*d(nu)/dr - 2*G*r^2*m*d^2(nu)/dr^2 + G*r*m*d(nu)/dr - 2*G*r*d(m)/dr + 2*G*m)/((c^2*r - 2*G*m)*r^2)

In [12]:
Ricci[2,2].expand().collect(nu.diff(r)).collect(nu.diff(r,r)).collect(c^2*exp(2*nu))

-r*d(nu)/dr + 2*G*m*d(nu)/dr/c^2 + G*d(m)/dr/c^2 + G*m/(c^2*r)

Ricci scalar is obtained by the `ricci_scalar()` method acting on `g`:

In [13]:
Ric_scalar = g.ricci_scalar()
(Ric_scalar.coord_function(X)).collect(nu.diff(r)).collect(nu.diff(r,r)).collect(c^2*exp(2*nu))

-2*(c^2*r^2*(d(nu)/dr)^2 + c^2*r^2*d^2(nu)/dr^2 + 2*c^2*r*d(nu)/dr - (2*r*m*(d(nu)/dr)^2 + 2*r*m*d^2(nu)/dr^2 + (r*d(m)/dr + 3*m)*d(nu)/dr + 2*d(m)/dr)*G)/(c^2*r^2)

It is the trace of the Ricci tensor, $R = R_\mu^\mu$: 

In [14]:
Ric_scalar == Ricci.up(g, 1).trace(0, 1)

True

Left side of the Einstein equations is

In [15]:
E = Ricci - (Ric_scalar*g)/2; E.display()

Ric(g)-unnamed metric = 2*G*e^(2*nu)*d(m)/dr/r^2 dt*dt + 2*(c^2*r^2*d(nu)/dr - (2*r*m*d(nu)/dr + m)*G)/(c^2*r^3 - 2*G*r^2*m) dr*dr + (c^2*r^3*(d(nu)/dr)^2 + c^2*r^3*d^2(nu)/dr^2 + c^2*r^2*d(nu)/dr - (2*r^2*m*(d(nu)/dr)^2 + 2*r^2*m*d^2(nu)/dr^2 + r*d(m)/dr + (r^2*d(m)/dr + r*m)*d(nu)/dr - m)*G)/(c^2*r) dth*dth - ((2*r^2*m*(d(nu)/dr)^2 + 2*r^2*m*d^2(nu)/dr^2 + r*d(m)/dr + (r^2*d(m)/dr + r*m)*d(nu)/dr - m)*G*sin(th)^2 - (c^2*r^3*(d(nu)/dr)^2 + c^2*r^3*d^2(nu)/dr^2 + c^2*r^2*d(nu)/dr)*sin(th)^2)/(c^2*r) dphi*dphi

Now for the energy-momentum tensor, $T_{\mu\nu}$: 

In [16]:
u = M.vector_field('u')
u[0] = exp(-nu)
u.display()

u = e^(-nu) d/dt

We can check if it is indeed the timelike 4-vector by checking $u_\mu u^\mu = -c^2$ by contracting it with the metric `g` using a method `contract()`: 

In [17]:
umuumu = g.contract(0,u,0).contract(0,u,0).coord_function(X)
umuumu == -c^2

True

The product $u_\mu u^\mu$ can be also obtained in much a simpler way, by just invoking

In [18]:
umuumu = g(u,u)
umuumu == -c^2

True

Let's now addopt $T_{\mu\nu}$ in perfect fluid form:  

In [19]:
u_form = u.down(g)
T = (rho + p/c^2)*(u_form*u_form) + p*g
T.set_name('T')
print T
T.display()

Field of symmetric bilinear forms T on the 4-dimensional differentiable manifold M


T = c^4*e^(2*nu)*rho dt*dt + c^2*r*P/(c^2*r - 2*G*m) dr*dr + r^2*P dth*dth + r^2*P*sin(th)^2 dphi*dphi

In [20]:
Ttrace = (T.up(g, 0)).trace(0, 1)
Ttrace.display()

M --> R
(t, r, th, phi) |--> -c^2*rho + 3*P

Three components of the Einstein equations are as follows - the $E_{tt}$ one: 

In [21]:
E0=(E[0,0] - (8*pi*G/c^4)*T[0,0]).expr() == 0

A small reorganization of the first equation, using the function solve() to solve for $dm/dr$: 

In [22]:
E0 = solve((E0*(-r^2/(G*exp(2*nu))/2)).expand().simplify(), m.diff(r))[0]

Using SageManifolds ExpressionNice to display the derivatives in textbook form: 

In [23]:
from sage.manifolds.utilities import ExpressionNice

In [24]:
ExpressionNice(E0)

d(m)/dr == 4*pi*r^2*rho

Radial component of Einstein's equations, $E_{rr}$: 

In [25]:
E1 = (E[1,1] - (8*pi*G/c^4)*T[1,1]).expr() == 0

In [26]:
E1 = solve((E1*(c^4*r^3 - 2*G*c^2*r^2*m)/2).expand().simplify_full(), nu.diff(r))[0]
ExpressionNice(E1)

d(nu)/dr == (4*pi*r^3*P + c^2*m)*G/(c^4*r^2 - 2*G*c^2*r*m)

For the third equation we use radial part of the energy-momentum conservation equation $\nabla_\mu T^{\mu\nu}$. First, to define the energy-momentum tensor $T^{\mu\nu}$ itself:

In [27]:
Tup = T.up(g,0).up(g,1)
Tup[:]

[                 e^(-2*nu(r))*rho(r)                                    0                                    0                                    0]
[                                   0 (c^2*r*P(r) - 2*G*P(r)*m(r))/(c^2*r)                                    0                                    0]
[                                   0                                    0                             P(r)/r^2                                    0]
[                                   0                                    0                                    0                 P(r)/(r^2*sin(th)^2)]

Connection ${\tt nab}$ for the covariant derivative, and the printout of the non-vanishing Christoffel symbols:  

In [28]:
nab = g.connection()
nab.display()

Gam^t_t,r = d(nu)/dr 
Gam^t_r,t = d(nu)/dr 
Gam^r_t,t = (c^2*r*e^(2*nu)*d(nu)/dr - 2*G*e^(2*nu)*m*d(nu)/dr)/r 
Gam^r_r,r = (r*d(m)/dr - m)*G/(c^2*r^2 - 2*G*r*m) 
Gam^r_th,th = -(c^2*r - 2*G*m)/c^2 
Gam^r_phi,phi = -(c^2*r*sin(th)^2 - 2*G*m*sin(th)^2)/c^2 
Gam^th_r,th = 1/r 
Gam^th_th,r = 1/r 
Gam^th_phi,phi = -cos(th)*sin(th) 
Gam^phi_r,phi = 1/r 
Gam^phi_th,phi = cos(th)/sin(th) 
Gam^phi_phi,r = 1/r 
Gam^phi_phi,th = cos(th)/sin(th)

In [29]:
co = nab(Tup)

The following calculates the radial component of $\nabla_\mu T^{\mu\nu}$: 

In [30]:
cosum = 0
# radial component of the covariant derivative: 
for i in M.irange():
    cosum += co[i,1,i]
cosum

(c^2*r*d(P)/dr - 2*(m*d(P)/dr + (c^2*m*rho + P*m)*d(nu)/dr)*G + (c^4*r*rho + c^2*r*P)*d(nu)/dr)/(c^2*r)

Solve for $dp/dr$: 

In [31]:
E2 = solve(cosum.expr(), p.diff(r))[0]
ExpressionNice(E2)

d(P)/dr == -(c^2*rho + P)*d(nu)/dr

Finally, the three TOV equations: 

In [32]:
ExpressionNice(E0), ExpressionNice(E1), ExpressionNice(E2)

(d(m)/dr == 4*pi*r^2*rho,
 d(nu)/dr == (4*pi*r^3*P + c^2*m)*G/(c^4*r^2 - 2*G*c^2*r*m),
 d(P)/dr == -(c^2*rho + P)*d(nu)/dr)